<a href="https://colab.research.google.com/github/dave20874/RapidReact-Trajectory/blob/main/RapidReact_Trajectory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

In [ ]:
in_to_m = 25.4/1000
flywheel_r = 2.0 * in_to_m
high_goal_h = (8*12+8) * in_to_m  # 8' 8" Goal
low_goal_h = (3*12+5) * in_to_m   # 3' 5" Goal
shooter_h = 16.0 * in_to_m        # 16" height of shooter
gravity = 9.8                     # m/s^2
m_to_ft = 1000.0/25.4/12.0
shooter_loss = 0.575
CRITICAL_ANGLE = math.radians(-30.0)

def launch_v(rpm):
  v_mps = rpm/60.0*flywheel_r*2.0*math.pi  # rev/min / sec/min * m -> m/sec
  return v_mps*shooter_loss

# Returns None if shot is invalid
# Returns (dist, arr_angle, max_height, rpm, trajectory) if shot is valid
def shoot(rpm, angle_rad, goal_h=high_goal_h):
  v0 = launch_v(rpm)
  # print(f"v0 = {v0}")
  vx = v0 * math.cos(angle_rad)
  vy = v0 * math.sin(angle_rad)

  x = 0.0
  y = shooter_h

  h = 0.001     # 1 ms steps

  # Advance trajectory 1ms at a time until it is descending below goal height
  trajectory = []
  trajectory.append((x, y))
  h_max = y
  while (vy > 0.0) | (y > goal_h):
    v_new = vy - h*gravity
    x += vx*h
    y += (v_new+vy)/2.0 * h
    vy = v_new

    if y > h_max:
      h_max = y

    trajectory.append((x, y))

  arr_angle = math.atan2(vy, vx)

  retval = (x, arr_angle, h_max, rpm, trajectory)

  if h_max < goal_h:
    return None

  if arr_angle > CRITICAL_ANGLE:
    return None

  return retval

result = shoot(3000.0, math.radians(67.0))

print(f"dist:{result[0]*m_to_ft} ft, arr_ang:{math.degrees(result[1])} deg, max_h:{result[2]*m_to_ft} ft")

dist:16.445672978288634 ft, arr_ang:-55.684965573850796 deg, max_h:13.277173670299966 ft


In [ ]:
for angle_deg in range(65, 90):
  angle = math.radians(angle_deg)

  min_range = None
  max_range = None
  for rpm in range(2000, 4500, 10):
    soln = shoot(rpm, angle)
    if soln is not None:
      if (min_range is None):
        min_range = soln
      elif (soln[0] < min_range[0]):
        # replace min_range solution with this solution
        min_range = soln

      if (max_range is None):
        max_range = soln
      elif (soln[0] > max_range[0]):
        # replace max_range solution with this solution
        max_range = soln

  # report min, max range solutions
  print(f"Angle: {angle_deg} deg:")
  print(f"  min: {min_range[0]*m_to_ft}, arr: {math.degrees(min_range[1])} deg, max_h: {min_range[2]*m_to_ft} ft")
  print(f"  max: {max_range[0]*m_to_ft}, arr: {math.degrees(max_range[1])} deg, max_h: {max_range[2]*m_to_ft} ft")

Angle: 65 deg:
  min: 9.38231493761819, arr: -30.205910089231004 deg, max_h: 9.245654831614438 ft
  max: 44.67533527773636, arr: -61.16604605421512 deg, max_h: 27.268761045427745 ft
Angle: 66 deg:
  min: 8.946874045337486, arr: -31.254402815507365 deg, max_h: 9.243391425500576 ft
  max: 43.399682392908716, arr: -62.34264215480058 deg, max_h: 27.684648765155682 ft
Angle: 67 deg:
  min: 8.347061315075079, arr: -30.964611811351162 deg, max_h: 9.169686494056787 ft
  max: 42.07919916483592, arr: -63.5342679685516 deg, max_h: 28.08766845202054 ft
Angle: 68 deg:
  min: 7.7323711717389525, arr: -30.06440765661218 deg, max_h: 9.088718073170167 ft
  max: 40.68028744275826, arr: -64.6990519776377 deg, max_h: 28.47732263872665 ft
Angle: 69 deg:
  min: 7.292743282990435, arr: -30.76427811515102 deg, max_h: 9.065594288505958 ft
  max: 39.22367031620267, arr: -65.85919149142246 deg, max_h: 28.853143742678984 ft
Angle: 70 deg:
  min: 6.836775930809902, arr: -31.069864938532405 deg, max_h: 9.0360676474